## 데이터 로드

In [1]:
# from google.colab import drive
# drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import pandas as pd
import numpy as np

# !pip install sktime
from sktime.forecasting.model_selection import temporal_train_test_split

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.9 MB 7.3 MB/s 


In [3]:
data_file = "/content/gdrive/MyDrive/Colab Notebooks/AIFT/data.csv"
df = pd.read_csv(data_file,index_col='Unnamed: 0')
df = df.iloc[1:][:]
print(df.shape)
df.head()

(14145, 23)


,ma_w_x,macd_w_x,macdsignal_w_x,macdhist_w_x,rsi_w_x,ad_w_x,ts_end_x,ts_start_x,is_higher_x,offset_intra_day_x,...,macdsignal_w_y,macdhist_w_y,rsi_w_y,ad_w_y,ts_end_y,ts_start_y,is_higher_y,offset_intra_day_y,target_y,label
2021-12-02 11:12:00,32.906667,8.622361,22.457295,-13.834934,-1.847265,51515.668116,False,False,True,0.330769,...,-2.542392,1.449403,1.920541,-541806.0,False,False,False,0.330769,0.998628,NOP
2021-12-02 11:15:00,35.760000,2.547207,18.475277,-15.928070,-5.503149,25495.709956,False,False,True,0.338462,...,-2.110002,1.729559,7.159809,-309922.4,False,False,False,0.338462,0.997340,NOP
2021-12-02 11:18:00,42.043333,-2.153807,14.349460,-16.503267,-4.424460,21430.047899,False,False,True,0.346154,...,-1.723365,1.546547,2.957516,-377094.0,False,False,False,0.346154,0.998507,NOP
2021-12-02 11:21:00,46.373333,-6.761681,10.127232,-16.888913,-4.671662,15676.338589,False,False,True,0.353846,...,-1.278042,1.781292,8.215010,-245900.2,False,False,False,0.353846,0.997179,X
2021-12-02 11:24:00,53.736667,-6.298201,6.842146,-13.140346,0.011823,24676.029280,False,False,True,0.361538,...,-0.888162,1.559519,4.020271,-663077.5,False,False,False,0.361538,0.998305,NOP


In [4]:
effective_cols = [

       'ma_w_x', 'macd_w_x', 'macdsignal_w_x', 'macdhist_w_x', 'rsi_w_x', 'ad_w_x', 'is_higher_x', 
       'ts_end_x', 'ts_start_x', 'offset_intra_day_x', 
       'ma_w_y', 'macd_w_y', 'macdsignal_w_y', 'macdhist_w_y', 'rsi_w_y', 'ad_w_y', 'is_higher_y', 


       'label'
       
       ]

In [5]:
data_df = df[effective_cols].dropna(axis=0)

## 데이터 셋 분할, 전처리

In [6]:
learn_split = data_df.loc[:'2022-09-30']
test_split = data_df.loc['2022-10-01':'2022-12-08']

In [7]:
X_train, X_val = temporal_train_test_split(learn_split, test_size=0.2)
X_test = test_split

In [8]:
y_train = X_train.pop('label')
y_val = X_val.pop('label')
y_test = X_test.pop('label')

In [9]:
y_train.value_counts(normalize=True)

NOP    0.910646
Y      0.048898
X      0.040456
Name: label, dtype: float64

In [10]:
y_val.value_counts(normalize=True)

NOP    0.943446
X      0.029811
Y      0.026743
Name: label, dtype: float64

In [11]:
y_test.value_counts(normalize=True)

NOP    0.912655
X      0.055512
Y      0.031832
Name: label, dtype: float64

In [30]:
# ts_end_x, ts_start_x
data_df.groupby('ts_end_x').count()

,ma_w_x,macd_w_x,macdsignal_w_x,macdhist_w_x,rsi_w_x,ad_w_x,is_higher_x,ts_start_x,offset_intra_day_x,ma_w_y,macd_w_y,macdsignal_w_y,macdhist_w_y,rsi_w_y,ad_w_y,is_higher_y,label
ts_end_x,,,,,,,,,,,,,,,,,
False,14145,14145,14145,14145,14145,14145,14145,14145,14145,14145,14145,14145,14145,14145,14145,14145,14145


In [31]:
data_df.groupby('ts_start_x').count()

,ma_w_x,macd_w_x,macdsignal_w_x,macdhist_w_x,rsi_w_x,ad_w_x,is_higher_x,ts_end_x,offset_intra_day_x,ma_w_y,macd_w_y,macdsignal_w_y,macdhist_w_y,rsi_w_y,ad_w_y,is_higher_y,label
ts_start_x,,,,,,,,,,,,,,,,,
False,14145,14145,14145,14145,14145,14145,14145,14145,14145,14145,14145,14145,14145,14145,14145,14145,14145


In [12]:
X_train.drop(columns=['ts_end_x','ts_start_x'],inplace=True)
X_val.drop(columns=['ts_end_x','ts_start_x'],inplace=True)
X_test.drop(columns=['ts_end_x','ts_start_x'],inplace=True)

X_train = X_train.astype({'is_higher_x':'bool','is_higher_y':'bool'})
X_val = X_val.astype({'is_higher_x':'bool','is_higher_y':'bool'})
X_test = X_test.astype({'is_higher_x':'bool','is_higher_y':'bool'})

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


ts_end_x, ts_start_x 값이 전부 False로 

In [13]:
# imbalanced
from imblearn.over_sampling import RandomOverSampler
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_resampled, y_resampled = smt.fit_resample(X_train, y_train)

from sklearn.model_selection import train_test_split
X_resampled_train, X_resampled_val, y_resampled_train, y_resampled_val = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

In [14]:
y_resampled_train.value_counts(normalize=True)

Y      0.341890
X      0.340428
NOP    0.317682
Name: label, dtype: float64

In [15]:
y_resampled_val.value_counts(normalize=True)

X      0.348223
Y      0.330215
NOP    0.321562
Name: label, dtype: float64

## flaml으로 최적 Hyperparameter 가져오기

In [38]:
# !pip install flaml
from flaml import AutoML
automl = AutoML()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 208 kB 7.3 MB/s 
     |████████████████████████████████| 2.0 MB 60.7 MB/s 
  Attempting uninstall: lightgbm
    Found existing installation: lightgbm 2.2.3
    Uninstalling lightgbm-2.2.3:
      Successfully uninstalled lightgbm-2.2.3


In [40]:
automl.fit(X_resampled_train, y_resampled_train, 
          task="classification", 
          X_val=X_resampled_val, y_val=y_resampled_val, 
          time_budget=60*10, 
          log_file_name='.segment_automl_baseline2_10m.log', 
          estimator_list=["xgboost"])

[flaml.automl: 12-11 17:25:54] {2599} INFO - task = classification
INFO:flaml.automl:task = classification
[flaml.automl: 12-11 17:25:54] {2601} INFO - Data split method: stratified
INFO:flaml.automl:Data split method: stratified
[flaml.automl: 12-11 17:25:54] {2604} INFO - Evaluation method: holdout
INFO:flaml.automl:Evaluation method: holdout
[flaml.automl: 12-11 17:25:54] {2726} INFO - Minimizing error metric: log_loss
INFO:flaml.automl:Minimizing error metric: log_loss
[flaml.automl: 12-11 17:25:54] {2870} INFO - List of ML learners in AutoML Run: ['xgboost']
INFO:flaml.automl:List of ML learners in AutoML Run: ['xgboost']
[flaml.automl: 12-11 17:25:54] {3166} INFO - iteration 0, current learner xgboost
INFO:flaml.automl:iteration 0, current learner xgboost
[flaml.automl: 12-11 17:25:54] {3296} INFO - Estimated sufficient time budget=2558s. Estimated necessary time budget=3s.
INFO:flaml.automl:Estimated sufficient time budget=2558s. Estimated necessary time budget=3s.
[flaml.automl

## Model 적합 & 결과

In [41]:
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
model = XGBClassifier(colsample_bylevel=0.7845109195865365,
                      colsample_bytree=0.6447843507910973,
                      grow_policy='lossguide',
                      learning_rate=0.2856559413650925,
                      max_depth=0,
                      max_leaves=203,
                      min_child_weight=0.3269890996192655,
                      n_estimators=696,
                      n_jobs=-1,
                      objective='multi:softprob',
                      reg_alpha=0.03134269983049553,
                      reg_lambda=0.004570510428599539,
                      subsample=1.0,
                      tree_method='hist',
                      use_label_encoder=False,
                      verbosity=0
                      )

In [42]:
model.fit(X_resampled_train, y_resampled_train, early_stopping_rounds=200,
           eval_metric='merror', eval_set=[(X_resampled_val, y_resampled_val)])

[0]	validation_0-merror:0.233396
Will train until validation_0-merror hasn't improved in 200 rounds.
[1]	validation_0-merror:0.169083
[2]	validation_0-merror:0.139149
[3]	validation_0-merror:0.119504
[4]	validation_0-merror:0.107109
[5]	validation_0-merror:0.097521
[6]	validation_0-merror:0.094715
[7]	validation_0-merror:0.0884
[8]	validation_0-merror:0.082788
[9]	validation_0-merror:0.07928
[10]	validation_0-merror:0.075772
[11]	validation_0-merror:0.068756
[12]	validation_0-merror:0.066885
[13]	validation_0-merror:0.065248
[14]	validation_0-merror:0.061506
[15]	validation_0-merror:0.059635
[16]	validation_0-merror:0.0587
[17]	validation_0-merror:0.056829
[18]	validation_0-merror:0.056829
[19]	validation_0-merror:0.052853
[20]	validation_0-merror:0.050515
[21]	validation_0-merror:0.047942
[22]	validation_0-merror:0.04724
[23]	validation_0-merror:0.046539
[24]	validation_0-merror:0.044668
[25]	validation_0-merror:0.044902
[26]	validation_0-merror:0.0442
[27]	validation_0-merror:0.04349

XGBClassifier(colsample_bylevel=0.7845109195865365,
              colsample_bytree=0.6447843507910973, grow_policy='lossguide',
              learning_rate=0.2856559413650925, max_depth=0, max_leaves=203,
              min_child_weight=0.3269890996192655, n_estimators=696, n_jobs=-1,
              objective='multi:softprob', reg_alpha=0.03134269983049553,
              reg_lambda=0.004570510428599539, subsample=1.0,
              tree_method='hist', use_label_encoder=False, verbosity=0)

In [43]:
'''compute predictions of testing dataset''' 
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)

In [44]:
pd.Series(y_pred).unique()

array(['NOP', 'Y', 'X'], dtype=object)

In [45]:
pd.Series(y_pred).value_counts(normalize=True)

NOP    0.887811
Y      0.063276
X      0.048913
dtype: float64

In [46]:
y_test.value_counts(normalize=True)

NOP    0.912655
X      0.055512
Y      0.031832
Name: label, dtype: float64

In [47]:
# confusion_matrix
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score
import seaborn as sns
 
cm = confusion_matrix(y_test,y_pred)
cm

array([[2085,  121,  145],
       [ 132,    4,    7],
       [  70,    1,   11]])

In [48]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

         NOP       0.91      0.89      0.90      2351
           X       0.03      0.03      0.03       143
           Y       0.07      0.13      0.09        82

    accuracy                           0.82      2576
   macro avg       0.34      0.35      0.34      2576
weighted avg       0.84      0.82      0.83      2576



In [49]:
# 모델 저장
'''pickle and save the automl object'''
import pickle
with open('/content/gdrive/MyDrive/Colab Notebooks/AIFT/model_xgboost.pkl', 'wb') as f:
    pickle.dump(model, f, pickle.HIGHEST_PROTOCOL)
'''load pickled automl object'''
with open('/content/gdrive/MyDrive/Colab Notebooks/AIFT/model_xgboost.pkl', 'rb') as f:
    model = pickle.load(f)